In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
import pydicom
import png
import csv
from os.path import join

In [ ]:
def convert_png(inputdata,savepath,index_num):
    shape = inputdata.shape
    image_2d = inputdata.astype(float)
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    image_2d_scaled = np.uint8(image_2d_scaled)
    
    with open(join(savepath + str(index_num) + '.png'), 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)

In [ ]:
def checkmask (dicdata):
    pd_dic = pd.DataFrame(dicdata)
    sum_zero = (pd_dic == 0).sum(axis=1).sum()
    sum_size = pd_dic.shape[0]*pd_dic.shape[1]
    percentage = sum_zero/sum_size
    return percentage

In [ ]:
def dcmtopng(input_data, folder_path, save_path):   
# input_data: csv file (should be read by pandas, type Dataframe), 
# folder_path: the path of dicom dataset, 
# save_path: the path to save png

    index_sum = len(input_data) # calculate the length of the dataset
    for i in range(index_sum):
        mask_path = input_data['ROI mask file path'][i].strip()  # strip the '/n' in the file path of csv file
        mass_path = input_data['cropped image file path'][i].strip()
        
        mask_path = join(folder_path + mask_path) # creat the complete path of file
        mass_path = join(folder_path + mass_path)
        
        data = pydicom.dcmread(mask_path).pixel_array  # read the image information of dicom file
        
        if checkmask(data) > 0.75:  # check the data and convert it into png, then save to the save path
            data = pydicom.dcmread(mass_path).pixel_array
            convert_png(data,save_path,i)
            
        else:
            convert_png(data,save_path,i)

In [ ]:
train_path = '.../mass_case_description_train_set.csv'
df = pd.read_csv(train_path)

In [ ]:
folder_path = '.../Desktop/Mass/Train/CBIS-DDSM/'
save_path = '.../Desktop/PNG/Train/'

In [ ]:
dcmtopng(df, folder_path, save_path)